# Strategy Comparison and Backtesting

This notebook compares different trading strategies across market regimes.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data.data_loader import DataLoader
from src.data.data_preprocessor import DataPreprocessor
from src.data.feature_engineer import FeatureEngineer
from src.regime_detection.gmm_detector import GMMDetector
from src.strategies.trend_following import TrendFollowingStrategy
from src.strategies.mean_reversion import MeanReversionStrategy
from src.strategies.volatility_breakout import VolatilityBreakoutStrategy
from src.strategies.backtester import Backtester
from src.utils.reporting import PerformanceReporter

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Prepare Data and Regimes

In [ ]:
# Load and prepare data
loader = DataLoader()
raw_data = loader.load_data('SPY', start_date='2015-01-01', end_date='2023-12-31')

preprocessor = DataPreprocessor()
clean_data = preprocessor.clean_data(raw_data)

engineer = FeatureEngineer()
features = engineer.create_features(clean_data)
regime_features = engineer.extract_regime_features(features)

# Detect regimes
detector = GMMDetector(n_regimes=3, random_state=42)
detector.fit(regime_features)
regimes = detector.predict(regime_features)

print(f"Data prepared: {len(features)} periods")
print(f"Regimes detected: {regimes.nunique()} unique regimes")

## 2. Initialize Strategies

In [ ]:
# Create strategy instances
strategies = [
    TrendFollowingStrategy(name='Trend Following'),
    MeanReversionStrategy(name='Mean Reversion'),
    VolatilityBreakoutStrategy(name='Volatility Breakout')
]

print("Strategies initialized:")
for strategy in strategies:
    print(f"  - {strategy.name}")

## 3. Run Backtests

In [ ]:
# Initialize backtester
backtester = Backtester(
    initial_capital=100000,
    commission=0.001,
    slippage=0.0005
)

# Run backtests
results = {}
for strategy in strategies:
    print(f"\nBacktesting {strategy.name}...")
    result = backtester.run(strategy, features, regime_labels=regimes)
    results[strategy.name] = result
    
    metrics = result['metrics']
    print(f"  Total Return: {metrics['total_return']:.2%}")
    print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"  Max Drawdown: {metrics['max_drawdown']:.2%}")
    print(f"  Win Rate: {metrics['win_rate']:.2%}")
    print(f"  Trades: {metrics['n_trades']}")

## 4. Compare Strategies

In [ ]:
# Create comparison table
comparison = backtester.compare_strategies(strategies, features, regimes)

print("\nStrategy Comparison:")
display(comparison)

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total return comparison
comparison['Total Return'].plot(kind='bar', ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Total Return', fontweight='bold')
axes[0,0].set_ylabel('Return')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)

# Sharpe ratio comparison
comparison['Sharpe'].plot(kind='bar', ax=axes[0,1], color='green')
axes[0,1].set_title('Sharpe Ratio', fontweight='bold')
axes[0,1].set_ylabel('Sharpe')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].axhline(y=1, color='r', linestyle='--', alpha=0.3, label='Threshold')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Max drawdown comparison
comparison['Max DD'].plot(kind='bar', ax=axes[1,0], color='red')
axes[1,0].set_title('Maximum Drawdown', fontweight='bold')
axes[1,0].set_ylabel('Drawdown')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# Win rate comparison
comparison['Win Rate'].plot(kind='bar', ax=axes[1,1], color='orange')
axes[1,1].set_title('Win Rate', fontweight='bold')
axes[1,1].set_ylabel('Win Rate')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Equity Curves

In [ ]:
# Plot equity curves
plt.figure(figsize=(15, 8))

for strategy_name, result in results.items():
    equity = result['equity_curve']
    plt.plot(equity.index, equity, label=strategy_name, linewidth=2)

# Add buy & hold benchmark
buy_hold = features['close'] / features['close'].iloc[0] * 100000
plt.plot(buy_hold.index, buy_hold, label='Buy & Hold', 
         linestyle='--', linewidth=2, alpha=0.7, color='black')

plt.title('Equity Curves Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Regime-Specific Performance

In [ ]:
# Analyze performance by regime
for strategy_name, result in results.items():
    print(f"\n{'='*60}")
    print(f"{strategy_name} - Performance by Regime")
    print('='*60)
    
    if 'regime_analysis' in result['metrics']:
        regime_analysis = result['metrics']['regime_analysis']
        
        regime_df = pd.DataFrame(regime_analysis).T
        display(regime_df)
        
        # Plot regime performance
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        regime_df['total_return'].plot(kind='bar', ax=axes[0], color='steelblue')
        axes[0].set_title(f'{strategy_name} - Returns by Regime', fontweight='bold')
        axes[0].set_ylabel('Total Return')
        axes[0].tick_params(axis='x', rotation=0)
        axes[0].grid(True, alpha=0.3)
        
        regime_df['sharpe'].plot(kind='bar', ax=axes[1], color='green')
        axes[1].set_title(f'{strategy_name} - Sharpe by Regime', fontweight='bold')
        axes[1].set_ylabel('Sharpe Ratio')
        axes[1].tick_params(axis='x', rotation=0)
        axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.3)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

## 7. Generate Reports

In [ ]:
# Generate detailed reports
reporter = PerformanceReporter(output_dir='../reports')

for strategy_name, result in results.items():
    report = reporter.generate_report(result, strategy_name)
    
    # Save text report
    filename = f"{strategy_name.lower().replace(' ', '_')}_report.txt"
    reporter.save_text_report(report, filename)
    print(f"Saved report: {filename}")

# Generate comparison report
reporter.compare_strategies(results, 'strategy_comparison.html')
print("\nComparison report saved")

## Next Steps

- Proceed to `04_performance_report.ipynb` for detailed analysis
- Reports generated in `reports/` directory
- Consider adaptive strategy selection based on regime